In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### EDA Notebook

#### Notebook Goal: Find fields of interest
#### Overall Goal: Find Key Factors for repeat conversions, a user using a Copilot recommendation at least 3 of last 4 weeks.

In [234]:
# Load in the requiste data:
account_df = pd.read_csv("../data/Product Insights Case Study Dataset - Account Level Dataset Case Study.csv")
user_df = pd.read_csv("../data/Product Insights Case Study Dataset - User Level Dataset Case Study.csv")

#### Account DF:

In [ ]:
account_df.columns

In [ ]:
account_df.info()

In [237]:
account_cols_of_interest = ["Account ID New", "Segment", "INDUSTRY", "Current Copilot Package", "Copilot First Access Date", "HAS_COPILOT", "HAS_SIGNALS", "# Purchased Seats", "# Power Users"]

In [238]:
# Only utilize necessary columns initially:
account_df = account_df[account_cols_of_interest]

In [ ]:
# Make sure all teams have Copilot or else exclude:
account_df["HAS_COPILOT"].value_counts()

Why are these columns null?

What is a power user exactly?

In [ ]:
print("Null Purchased Seats:", sum(account_df["# Purchased Seats"].isnull()))
print("Null Power Users:", sum(account_df["# Power Users"].isnull()))

#### User DF:

In [ ]:
user_df.columns

In [ ]:
# See how many unique users there are:
user_df["User ID"].nunique()


In [243]:
### Note: There is no Login Count 
user_cols_of_interest = ["User ID", "Account ID", "Copilot Current Package", "Segment", "Industry", "Sub Persona",
                         "Positioned for Success", "Attended Any Copilot Training", "Has Ta", "Conversion Cohort",
                         "Onboarding Project Onboarding Phase", "Funnel Step", "Conversion Rate Last 30 Days", "Viewed Account Updates Last 30 Days"]

In [244]:
user_df = user_df[user_cols_of_interest]

In [ ]:
user_df

In [ ]:
user_df["Copilot Current Package"].value_counts()

In [247]:
### We should exclude users without Copilot
user_df = user_df[user_df["Copilot Current Package"].notnull() & (user_df["Copilot Current Package"] != "No Copilot Package")]

In [ ]:
user_df["Has Ta"].value_counts()

In [249]:
### If there are no Target Accounts then we cannot recommend 
user_df = user_df[user_df["Has Ta"] == True]

In [250]:
user_df.reset_index(drop=True, inplace=True)

#### Recommendation at this point is first get Copilot, then set up Target Accounts: 

In [251]:
### Target Variable is supposed to be Funnel Step

In [ ]:
user_df.info()

#### Unsure why ther are still null Conversion Cohorts, but looks promising at this point:

In [ ]:

# Funnel Step is essentially the target variable: Returning Conversion is highest, Converted Account Update is next up, etc.
user_df["Funnel Step"].value_counts()

In [254]:
def plot_crosstab_bar_chart(df, row_var, col_var, filename=None, figsize=(10, 6)):
    """
    Generates a grouped bar chart from a cross-tabulation of two variables.

    Parameters:
    df (pd.DataFrame): The input DataFrame (e.g., user_df).
    row_var (str): The column name for the crosstab index (rows/x-axis groups).
    col_var (str): The column name for the crosstab columns (bar colors/legend).
    figsize (tuple): The size of the figure.

    Returns:
    tuple: (matplotlib.figure.Figure, matplotlib.axes.Axes)
    """
    # 1. Aggregate the data using crosstab
    cross_tab = pd.crosstab(df[row_var], df[col_var])

    # 2. Create the grouped bar chart using the fig, ax format
    fig, ax = plt.subplots(figsize=figsize)

    # Plot the crosstab data directly to the axes object
    cross_tab.plot(kind='bar', ax=ax)

    # 3. Add dynamic title and labels
    ax.set_title(f"Count of {col_var} by {row_var}")
    ax.set_xlabel(row_var)
    ax.set_ylabel("Count of Users")

    # Rotate x-axis labels for better readability
    ax.tick_params(axis='x', rotation=45)

    # Add a legend
    ax.legend(title=col_var)

    plt.tight_layout()

    # A. Generate the default filename if none is provided
    if filename is None:
        # Create a simple, snake_case-like name by replacing spaces with underscores
        clean_row_var = row_var.replace(" ", "_").lower()
        clean_col_var = col_var.replace(" ", "_").lower()
        
        # Construct the default filename
        generated_filename = f"{clean_row_var}_vs_{clean_col_var}_barchart.png"
        filename_to_save = generated_filename
    else:
        filename_to_save = filename

    # B. Save the file
    save_directory = os.path.join("..", "figs") 
    os.makedirs(save_directory, exist_ok=True)
    save_path = os.path.join(save_directory, filename_to_save)
    
    fig.savefig(save_path)
    print(f"Figure successfully saved to: {save_path}")

    return fig, ax

    return fig, ax

In [255]:
# Insert Target Variable:
target_steps = ["Returning Conversion", "Converted Account Update"]
user_df["converted_user"] = user_df["Funnel Step"].isin(target_steps)

In [256]:
semgent_cols = ["Copilot Current Package", "Sub Persona", "Positioned for Success",
                "Attended Any Copilot Training", "Industry", "Segment"]

In [ ]:
for segment_col in semgent_cols:
    plot_crosstab_bar_chart(user_df, segment_col, "Funnel Step")


In [ ]:
### It is much easier to read if not in a loop:
user_df.groupby("Copilot Current Package").agg({"converted_user": np.mean})


In [259]:
# Notes: Not much of a difference in package:

In [ ]:
user_df.groupby("Sub Persona").agg({"converted_user": np.mean})


In [261]:
# Notes: Interesting that Full Cycle Rep and Marketing Management are highest:

In [ ]:
user_df.groupby("Positioned for Success").agg({"converted_user": np.mean})


In [263]:
# Notes: Winner is recommend get them to Good

In [ ]:
user_df.groupby("Attended Any Copilot Training").agg({"converted_user": np.mean})

In [265]:
# Notes: Getting them Trained is notieable:

In [ ]:
user_df.groupby("Industry").agg({"converted_user": np.mean})

In [267]:
# Notes: More time would analyze the support of each of these

In [ ]:
user_df.groupby("Segment").agg({"converted_user": np.mean})

In [269]:
# Notes: Nothing specifically noticeable in this:

In [270]:
def plot_crosstab_heatmap(df, row_var, col_var, filename=None, figsize=(10, 6)):
    """
    Generates a heatmap showing the count distribution of two categorical variables.

    Parameters:
    df (pd.DataFrame): The input DataFrame (e.g., user_df).
    row_var (str): The column name for the heatmap Y-axis (rows).
    col_var (str): The column name for the heatmap X-axis (columns).
    figsize (tuple): The size of the figure.

    Returns:
    tuple: (matplotlib.figure.Figure, matplotlib.axes.Axes)
    """
    # 1. Aggregate the data using crosstab to get counts
    cross_tab = pd.crosstab(df[row_var], df[col_var])

    # 2. Create the figure and axes
    fig, ax = plt.subplots(figsize=figsize)

    # 3. Generate the heatmap using seaborn
    sns.heatmap(
        cross_tab,
        annot=True,      # Show the actual count numbers in each cell
        fmt="d",         # Format annotations as integers
        cmap="YlGnBu",   # Color palette (can be changed to 'viridis', 'magma', etc.)
        cbar_kws={'label': 'Count of Users'},
        linewidths=.5,   # Add thin lines between cells
        ax=ax
    )

    # 4. Set dynamic title and labels
    ax.set_title(f"Count of {col_var} by {row_var}")
    ax.set_xlabel(col_var) # X-axis title comes from the cross_tab columns
    ax.set_ylabel(row_var) # Y-axis title comes from the cross_tab index

    # Ensure labels are visible (especially important for x-ticks in heatmaps)
    ax.tick_params(axis='x', rotation=0)
    ax.tick_params(axis='y', rotation=0)

    plt.tight_layout()

    # A. Generate the default filename if none is provided
    if filename is None:
        # Create a simple, snake_case-like name by replacing spaces with underscores
        clean_row_var = row_var.replace(" ", "_").lower()
        clean_col_var = col_var.replace(" ", "_").lower()
        
        # Construct the default filename
        generated_filename = f"{clean_row_var}_vs_{clean_col_var}_heatmap.png"
        filename_to_save = generated_filename
    else:
        filename_to_save = filename

    # B. Save the file
    save_directory = os.path.join("..", "figs") 
    os.makedirs(save_directory, exist_ok=True)
    save_path = os.path.join(save_directory, filename_to_save)
    
    fig.savefig(save_path)
    print(f"Figure successfully saved to: {save_path}")

    return fig, ax

    return fig, ax

In [ ]:
for segment_col in semgent_cols:
    plot_crosstab_heatmap(user_df, segment_col, "converted_user");

### Stage all of the data for Hypothesis Work:

In [272]:
account_df.to_csv("../data/account_df.csv",index=False)

In [273]:
user_df.to_csv("../data/user_df.csv",index=False)

In [274]:
merged_df = user_df.merge(account_df, left_on="Account ID", right_on="Account ID New", how="inner")\
                   .drop(columns=["Segment_y", "INDUSTRY", "Account ID New", "Current Copilot Package", "HAS_COPILOT"])\
                   .rename(columns={"Segment_x": "Segment"})

In [277]:
merged_df.to_csv("../data/merged_df.csv",index=False)

In [ ]:
merged_df